In [1]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=323d2f31686d6c66c46cea75ec45be7479deee663cb6e20176c1ddf92a3ad783
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import findspark
findspark.init()


In [3]:
from pyspark import SparkContext

In [4]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("CrimeDataAnalysis") \
    .getOrCreate()

# Now you can use the 'spark' variable to read CSV files or perform other Spark operations


In [5]:
!wget https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/

--2024-05-14 02:11:30--  https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [   <=>              ] 771.94K  1.03MB/s    in 0.7s    

2024-05-14 02:11:33 (1.03 MB/s) - ‘index.html’ saved [790469]



In [6]:
from pyspark.sql.functions import to_timestamp,col,lit
read_csv = spark.read.csv('sample_data/crime.csv', header=True).withColumn('DATE  OF OCCURRENCE', to_timestamp(col('DATE  OF OCCURRENCE'), 'MM/dd/yyyy hh:mm:ss a'))
read_csv.show(5)


+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|DATE  OF OCCURRENCE|               BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|JG497095|2023-11-08 20:50:00| 025XX N KEDZIE BLVD| 0810|               THEFT|             OVER $500|               STREET|     N|       N|1414|  35|    06|     1154609|     1916759|41.927407329| -87.70729439|(41.927407329, -8...|
|JG496991|2023-11-08 15:14:00| 0000X W CHICAGO AVE| 0560|             ASSAUL

**Schema Handling**

In [7]:
read_csv.printSchema()

root
 |-- CASE#: string (nullable = true)
 |-- DATE  OF OCCURRENCE: timestamp (nullable = true)
 |-- BLOCK: string (nullable = true)
 |--  IUCR: string (nullable = true)
 |--  PRIMARY DESCRIPTION: string (nullable = true)
 |--  SECONDARY DESCRIPTION: string (nullable = true)
 |--  LOCATION DESCRIPTION: string (nullable = true)
 |-- ARREST: string (nullable = true)
 |-- DOMESTIC: string (nullable = true)
 |-- BEAT: string (nullable = true)
 |-- WARD: string (nullable = true)
 |-- FBI CD: string (nullable = true)
 |-- X COORDINATE: string (nullable = true)
 |-- Y COORDINATE: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- LOCATION: string (nullable = true)



In [8]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType

In [9]:
read_csv.columns

['CASE#',
 'DATE  OF OCCURRENCE',
 'BLOCK',
 ' IUCR',
 ' PRIMARY DESCRIPTION',
 ' SECONDARY DESCRIPTION',
 ' LOCATION DESCRIPTION',
 'ARREST',
 'DOMESTIC',
 'BEAT',
 'WARD',
 'FBI CD',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION']

In [10]:
labels= [('CASE#',StringType()),
('DATE  OF OCCURRENCE',TimestampType()),
('BLOCK',StringType()),
(' IUCR',StringType()),
(' PRIMARY DESCRIPTION',StringType()),
(' SECONDARY DESCRIPTION',StringType()),
(' LOCATION DESCRIPTION',StringType()),
('ARREST',StringType()),
('DOMESTIC', BooleanType()),
('BEAT', StringType()),
('WARD',StringType()),
('FBI CD', StringType()),
('X COORDINATE',StringType()),
('Y COORDINATE',StringType()),
('LATITUDE', DoubleType()),
('LONGITUDE', DoubleType()),
('LOCATION', StringType())]

In [11]:
schema = StructType([StructField(x[0],x[1],True) for x in labels])


In [12]:
csv_df = spark.read.csv('sample_data/crime.csv', schema = schema)

In [13]:
csv_df.printSchema()

root
 |-- CASE#: string (nullable = true)
 |-- DATE  OF OCCURRENCE: timestamp (nullable = true)
 |-- BLOCK: string (nullable = true)
 |--  IUCR: string (nullable = true)
 |--  PRIMARY DESCRIPTION: string (nullable = true)
 |--  SECONDARY DESCRIPTION: string (nullable = true)
 |--  LOCATION DESCRIPTION: string (nullable = true)
 |-- ARREST: string (nullable = true)
 |-- DOMESTIC: boolean (nullable = true)
 |-- BEAT: string (nullable = true)
 |-- WARD: string (nullable = true)
 |-- FBI CD: string (nullable = true)
 |-- X COORDINATE: string (nullable = true)
 |-- Y COORDINATE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)



In [14]:
csv_df.show(5)

+--------+-------------------+-------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|DATE  OF OCCURRENCE|              BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+-------------------+-------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|               NULL|              BLOCK| IUCR| PRIMARY DESCRIPTION|   SECONDARY DESCRI...|  LOCATION DESCRIP...|ARREST|    NULL|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|        NULL|         NULL|            LOCATION|
|JG497095|               NULL|025XX N KEDZIE BLVD| 0810|               THEFT|   

**Showing column**

In [15]:
read_csv.select(' IUCR').show(5)

+-----+
| IUCR|
+-----+
| 0810|
| 0560|
| 051A|
| 0820|
| 0810|
+-----+
only showing top 5 rows



In [16]:
read_csv.select(col(' IUCR')).show(5)

+-----+
| IUCR|
+-----+
| 0810|
| 0560|
| 051A|
| 0820|
| 0810|
+-----+
only showing top 5 rows



**Showing multiple cols**

In [17]:
read_csv.select(' IUCR', ' PRIMARY DESCRIPTION', 'DOMESTIC').show(5)

+-----+--------------------+--------+
| IUCR| PRIMARY DESCRIPTION|DOMESTIC|
+-----+--------------------+--------+
| 0810|               THEFT|       N|
| 0560|             ASSAULT|       N|
| 051A|             ASSAULT|       N|
| 0820|               THEFT|       N|
| 0810|               THEFT|       N|
+-----+--------------------+--------+
only showing top 5 rows



**Add new column named NEW COL with all entries 1s**

In [18]:
from pyspark.sql.functions import lit

In [19]:
read_csv.withColumn('NEW COLUMN', lit(1)).show(5)

+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+----------+
|   CASE#|DATE  OF OCCURRENCE|               BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|NEW COLUMN|
+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+----------+
|JG497095|2023-11-08 20:50:00| 025XX N KEDZIE BLVD| 0810|               THEFT|             OVER $500|               STREET|     N|       N|1414|  35|    06|     1154609|     1916759|41.927407329| -87.70729439|(41.927407329, -8...|         1|
|JG496991|2023-11-08 15:14:00| 0

**Dropping column ` IUCR`**

In [20]:
read_csv.drop(' IUCR').show(5)

+--------+-------------------+--------------------+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|DATE  OF OCCURRENCE|               BLOCK| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+-------------------+--------------------+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|JG497095|2023-11-08 20:50:00| 025XX N KEDZIE BLVD|               THEFT|             OVER $500|               STREET|     N|       N|1414|  35|    06|     1154609|     1916759|41.927407329| -87.70729439|(41.927407329, -8...|
|JG496991|2023-11-08 15:14:00| 0000X W CHICAGO AVE|             ASSAULT|                SIMPLE|     

#### WORKING WITH ROWS

In [21]:
read_csv.select('DATE  OF OCCURRENCE').show(5)

+-------------------+
|DATE  OF OCCURRENCE|
+-------------------+
|2023-11-08 20:50:00|
|2023-11-08 15:14:00|
|2023-11-08 22:55:00|
|2024-03-07 14:15:00|
|2024-03-07 04:53:00|
+-------------------+
only showing top 5 rows



Filtering on Date

In [22]:
one_day_crime_df = read_csv.filter(col('DATE  OF OCCURRENCE').contains('2023-11-01'))
one_day_crime_df.count()



755

In [23]:
one_day_crime_df.show(5)

+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|DATE  OF OCCURRENCE|               BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|JG503603|2023-11-01 21:00:00|   024XX W FITCH AVE| 0930| MOTOR VEHICLE THEFT|  THEFT / RECOVERY ...| VEHICLE NON-COMME...|     N|       N|2411|  50|    07|     1158863|     1947386|42.011363066|-87.690818628|(42.011363066, -8...|
|JG518441|2023-11-01 07:00:00|044XX S DR MARTIN...| 1562|         SEX OFFENS

In [24]:
one_day_crime_df.count()

755

In [25]:
read_csv.count()

156171

In [26]:
new_csv = read_csv.union(one_day_crime_df)

In [27]:
new_csv.count()

156926

In [28]:
new_csv.orderBy('DATE  OF OCCURRENCE',ascending = False ).show(5)

+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|   CASE#|DATE  OF OCCURRENCE|               BLOCK| IUCR| PRIMARY DESCRIPTION| SECONDARY DESCRIPTION| LOCATION DESCRIPTION|ARREST|DOMESTIC|BEAT|WARD|FBI CD|X COORDINATE|Y COORDINATE|    LATITUDE|    LONGITUDE|            LOCATION|
+--------+-------------------+--------------------+-----+--------------------+----------------------+---------------------+------+--------+----+----+------+------------+------------+------------+-------------+--------------------+
|JH220340|2024-04-10 00:00:00|    019XX W 103RD ST| 1130|  DECEPTIVE PRACTICE|  FRAUD OR CONFIDEN...|           RESTAURANT|     N|       N|2213|  19|    11|     1165343|     1836362|41.706567182|-87.670131443|(41.706567182, -8...|
|JH220281|2024-04-10 00:00:00|056XX S PRINCETON...| 1310|     CRIMINAL DAMAG

In [29]:
crime_type = read_csv.select(' PRIMARY DESCRIPTION').distinct()

In [30]:
crime_type.count()


31

In [31]:
crime_type_list = crime_type.select(' PRIMARY DESCRIPTION').collect()
crime_list = [row[' PRIMARY DESCRIPTION'] for row in crime_type_list]
print(crime_list)

['OFFENSE INVOLVING CHILDREN', 'CRIMINAL SEXUAL ASSAULT', 'STALKING', 'PUBLIC PEACE VIOLATION', 'OBSCENITY', 'ARSON', 'GAMBLING', 'CRIMINAL TRESPASS', 'ASSAULT', 'LIQUOR LAW VIOLATION', 'MOTOR VEHICLE THEFT', 'THEFT', 'BATTERY', 'ROBBERY', 'HOMICIDE', 'PUBLIC INDECENCY', 'HUMAN TRAFFICKING', 'INTIMIDATION', 'PROSTITUTION', 'DECEPTIVE PRACTICE', 'CONCEALED CARRY LICENSE VIOLATION', 'SEX OFFENSE', 'CRIMINAL DAMAGE', 'NARCOTICS', 'NON-CRIMINAL', 'OTHER OFFENSE', 'KIDNAPPING', 'BURGLARY', 'WEAPONS VIOLATION', 'INTERFERENCE WITH PUBLIC OFFICER', 'OTHER NARCOTIC VIOLATION']


**Top 10 number of reported crimes by Primary type in descending order of occurance**

In [34]:
new_csv.groupBy(' PRIMARY DESCRIPTION').count().orderBy('count', ascending=False).show(10)

+--------------------+-----+
| PRIMARY DESCRIPTION|count|
+--------------------+-----+
|               THEFT|35272|
|             BATTERY|26718|
|     CRIMINAL DAMAGE|17691|
| MOTOR VEHICLE THEFT|15916|
|             ASSAULT|13826|
|       OTHER OFFENSE| 9932|
|  DECEPTIVE PRACTICE| 9656|
|             ROBBERY| 7010|
|   WEAPONS VIOLATION| 4779|
|            BURGLARY| 4503|
+--------------------+-----+
only showing top 10 rows



Percentage of Reported Crime that resulted in arrest


In [36]:
new_csv.select('ARREST').distinct().show()

+------+
|ARREST|
+------+
|     Y|
|     N|
+------+



In [37]:
new_csv.filter(col('ARREST')=='Y').count()/new_csv.select('ARREST').count()

0.1255432496845647

Top 3 locations for reported crimes

In [39]:
new_csv.groupBy(' LOCATION DESCRIPTION').count().orderBy('count', ascending=False).show(3)

+---------------------+-----+
| LOCATION DESCRIPTION|count|
+---------------------+-----+
|               STREET|44835|
|            APARTMENT|29902|
|            RESIDENCE|18501|
+---------------------+-----+
only showing top 3 rows

